# Setup
- Install alphafold3, colabfold
- Copy model weights from Google Drive (adjust path & give permission when asked)

In [1]:
%%shell
pip install alphafold3-polymer-bonds
ALPHAFOLD3_VERSION="2e2ffc1⁠"
source <(curl -fsSL https://raw.githubusercontent.com/jurgjn/alphafold3-polymer-bonds/refs/heads/main/colab/install)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00
Installing alphafold3 2e2ffc1⁠
Installing colabfold


In [2]:
# Copy weights from Google Drive to the default location at /root/models/af3.bin.zst
from google.colab import drive
drive.mount('/mnt/gdrive')
!mkdir -p /root/models/
!cp /mnt/gdrive/MyDrive/alphafold3/models/af3.bin.zst /root/models/af3.bin.zst
!md5sum /root/models/af3.bin.zst
drive.flush_and_unmount()

Mounted at /mnt/gdrive
ea076e2d05168d66a3d9a3fa62e029d0  /root/models/af3.bin.zst


In [3]:
# Standard imports
import argparse, collections, copy, gzip, json, os, os.path, re, string, sys
import numpy as np
# Helper functions for manipulating AlphaFold 3 input jsons: https://github.com/google-deepmind/alphafold3/blob/main/docs/input.md
from alphafold3_polymer_bonds.alphafold3_io import colab_data_pipeline, init_input_json, read_input_json, print_input_json, write_input_json

# Running inference on a single monomer

In [6]:
# Download sequences for a selection of species
%%shell
mkdir -p fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/Q8U2C1.fasta | sed -e "1s/.*/>pyrfu/" > fasta/pyrfu.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/P65845.fasta | sed -e "1s/.*/>ecoli/" > fasta/ecoli.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/P31115.fasta | sed -e "1s/.*/>yeast/" > fasta/yeast.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/Q09524.fasta | sed -e "1s/.*/>caeel/" > fasta/caeel.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/Q9W282.fasta | sed -e "1s/.*/>drome/" > fasta/drome.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/Q6P815.fasta | sed -e "1s/.*/>xentr/" > fasta/xentr.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/Q7ZW97.fasta | sed -e "1s/.*/>danre/" > fasta/danre.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/Q9JI38.fasta | sed -e "1s/.*/>mouse/" > fasta/mouse.fasta
curl -fsSL https://rest.uniprot.org/uniprotkb/Q9BZE2.fasta | sed -e "1s/.*/>human/" > fasta/human.fasta
ls -l fasta

total 36
-rw-r--r-- 1 root root 416 Oct  1 12:13 caeel.fasta
-rw-r--r-- 1 root root 475 Oct  1 12:13 danre.fasta
-rw-r--r-- 1 root root 515 Oct  1 12:13 drome.fasta
-rw-r--r-- 1 root root 282 Oct  1 12:13 ecoli.fasta
-rw-r--r-- 1 root root 497 Oct  1 12:13 human.fasta
-rw-r--r-- 1 root root 497 Oct  1 12:13 mouse.fasta
-rw-r--r-- 1 root root 278 Oct  1 12:13 pyrfu.fasta
-rw-r--r-- 1 root root 521 Oct  1 12:13 xentr.fasta
-rw-r--r-- 1 root root 457 Oct  1 12:13 yeast.fasta


In [ ]:
# Show sequence for one, e.g. drosophila
!cat fasta/drome.fasta

In [ ]:
# Use init_input_json() to create an AlphaFold 3 input for a single peptide sequence
monomer_example = init_input_json("""
MSATNDKKVVINKRLKGLSREALEKLTQTELIDKVIQLEAYNFQLRNLLQKKLSEKDKHD
KEYSGLIGNEAEGKVSQVAKTSSKVQKIRKFDWSSAHKRHVLLKITYFGWDYQGFACQED
SNDTIESNLFRALARTCLIESRATSNYHRCGRTDKEVSAFCQVISIDLRSKHPPESQLDP
TALSSEIDYCGLLNRVLPKNIQCVAWMPLRSPVYSARFDCVSRTYRYYFPKGDLDIAAMR
KACDLLVRHADFRNFCKMDVHNGVTNYMRNLQSARVEACDQTNHTNSGYDMYYLEIQANA
FLWHQIRCIMAVLLLVGQKKENPGVISDLLDVESNPCKPQYTPAIGLPLNLFRCDFRDHT
TRSVNHPSSGDADEEAMDTAADESNDLNAPEHLERDLTAWIYNEENLQKLIENTQCEWTQ
FSVKSTMIRNVLQQLENLFEENFKPKEKVLAQVILLQDSVNPRQYQPLLERKRCESLENR
IEHFVKKQRLIVKNETETE""".replace('\n', ''))
monomer_example

In [ ]:
# Google Colab does not have enough disk space to run the default AlphaFold 3 data pipeline (MSA + template search)
# We work around this by using the ColabFold MSA server
monomer_example_data = colab_data_pipeline(monomer_example)

In [ ]:
# AlphaFold 3 data pipeline encodes MSAs in a single json line, the output is unreadable
# Try using print_input_json() on the data pipeline output instead
monomer_example_data

In [ ]:
# AlphaFold 3 uses the name field in the input .json to generate the output file name
monomer_example_data['name'] = 'drome'
write_input_json(monomer_example_data, 'monomer_input/{}.json')
!ls -l monomer_input/

In [ ]:
# run_alphafold3_autogpu.sh runs the structure prediction step with settings adjusted for the available GPU
# --norun_data_pipeline turns off the default MSA/template search (as we're using the ColabFold MSA server)
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/monomer_input \
  --output_dir=/content/monomer_output \
  --norun_data_pipeline

In [ ]:
# Results from the prediction step should now appear under:
!ls -l /content/monomer_output/monomer_example

# Mouse-human dimer

In [7]:
# As an example pair, let's look up the sequences for mouse and human
!cat fasta/mouse.fasta
!cat fasta/yeast.fasta

>mouse
MAENTDRNQIEKLLNRVKELEQEVERLKKKKEQANNIKDSSIRENSLGSGKAKRAFDFSA
HGRRHVALKIAYLGWGYQGFASQENTSNTIEEKLFEALTKTRLVESRQTSNYHRCGRTDK
GVSAFGQVISLDLRSQFPTSRDSEDSNLKHEADDLAKEIRYTHILNRVLPADIRVLAWAP
VEPSFSARFSCLERTYRYFFPRADLDIATMNYAAQKYVGTHDFRNLCKMDVANGVINFQR
TILCAQVQLVAQSPGEERRQEPFQLCQFEVIGQAFLYHQVRCMMAILFLIGQGMEKPEII
DELLNIQKNPQKPQYSMAVEFPLVLYDCKFENTKWIYDHEVQEFNVTHLQQLWANHAVKT
HMLYSMLQGLDSVMVTCAAGTKMDEATEWRNIQPPVIKHTSAFVEGVKMRTYKPLMDRPK
CQGLESRIRHFVSRGRIEHPHLLHKEEIKARRDCADKEENTVVENPSKRVCIIDAEINSI
A
>yeast
MSNFIRRLVGKMKAISTGTNAIVSKKDSIYANWSKEQLIRRITELENANKPHSEKFQHIE
DNKKRKISQEEVTRSKAKKAPKKFDFSKHNTRFIALRFAYLGWNYNGLAVQKEYTPLPTV
EGTILEAMNKCKLVPSMVLQDYKFSRCGRTDKGVSAMNQVISLEVRSNLTDEEQRDPTND
SREIPYVHVLNQLLPDDIRISAVCLRPPPNFDARFSCVHRHYKYIFNGKNLNIEKMSKAA
SYFVGERDFRNFCKLDGSKQITNFKRTIISSKILPLSETFYCFDLVGSAFLWHQVRCMMA
ILFLVGQSLEVPEIVLRLTDIEKTPQRPVYEMANDIPLLLYDCKFPEMDWQEPTVDDYKA
IKFTTATEALTLHYELKAAVCNIFKDVLPTANTNNFSKTIINLGDGRGKVVGTYVKLEDR
SVMEPVEVVNAKYSKKKNNKNK


In [8]:
# init_input_json() can handle multiple sequences as input
mouse_yeast = init_input_json(
"""
MAENTDRNQIEKLLNRVKELEQEVERLKKKKEQANNIKDSSIRENSLGSGKAKRAFDFSA
HGRRHVALKIAYLGWGYQGFASQENTSNTIEEKLFEALTKTRLVESRQTSNYHRCGRTDK
GVSAFGQVISLDLRSQFPTSRDSEDSNLKHEADDLAKEIRYTHILNRVLPADIRVLAWAP
VEPSFSARFSCLERTYRYFFPRADLDIATMNYAAQKYVGTHDFRNLCKMDVANGVINFQR
TILCAQVQLVAQSPGEERRQEPFQLCQFEVIGQAFLYHQVRCMMAILFLIGQGMEKPEII
DELLNIQKNPQKPQYSMAVEFPLVLYDCKFENTKWIYDHEVQEFNVTHLQQLWANHAVKT
HMLYSMLQGLDSVMVTCAAGTKMDEATEWRNIQPPVIKHTSAFVEGVKMRTYKPLMDRPK
CQGLESRIRHFVSRGRIEHPHLLHKEEIKARRDCADKEENTVVENPSKRVCIIDAEINSI
A""".replace('\n', ''),
"""
MSNFIRRLVGKMKAISTGTNAIVSKKDSIYANWSKEQLIRRITELENANKPHSEKFQHIE
DNKKRKISQEEVTRSKAKKAPKKFDFSKHNTRFIALRFAYLGWNYNGLAVQKEYTPLPTV
EGTILEAMNKCKLVPSMVLQDYKFSRCGRTDKGVSAMNQVISLEVRSNLTDEEQRDPTND
SREIPYVHVLNQLLPDDIRISAVCLRPPPNFDARFSCVHRHYKYIFNGKNLNIEKMSKAA
SYFVGERDFRNFCKLDGSKQITNFKRTIISSKILPLSETFYCFDLVGSAFLWHQVRCMMA
ILFLVGQSLEVPEIVLRLTDIEKTPQRPVYEMANDIPLLLYDCKFPEMDWQEPTVDDYKA
IKFTTATEALTLHYELKAAVCNIFKDVLPTANTNNFSKTIINLGDGRGKVVGTYVKLEDR
SVMEPVEVVNAKYSKKKNNKNK""".replace('\n', ''),
)
mouse_yeast

OrderedDict([('dialect', 'alphafold3'),
             ('version', 2),
             ('name', 'name'),
             ('sequences',
              [OrderedDict([('protein',
                             OrderedDict([('id', 'A'),
                                          ('sequence',
                                           'MAENTDRNQIEKLLNRVKELEQEVERLKKKKEQANNIKDSSIRENSLGSGKAKRAFDFSAHGRRHVALKIAYLGWGYQGFASQENTSNTIEEKLFEALTKTRLVESRQTSNYHRCGRTDKGVSAFGQVISLDLRSQFPTSRDSEDSNLKHEADDLAKEIRYTHILNRVLPADIRVLAWAPVEPSFSARFSCLERTYRYFFPRADLDIATMNYAAQKYVGTHDFRNLCKMDVANGVINFQRTILCAQVQLVAQSPGEERRQEPFQLCQFEVIGQAFLYHQVRCMMAILFLIGQGMEKPEIIDELLNIQKNPQKPQYSMAVEFPLVLYDCKFENTKWIYDHEVQEFNVTHLQQLWANHAVKTHMLYSMLQGLDSVMVTCAAGTKMDEATEWRNIQPPVIKHTSAFVEGVKMRTYKPLMDRPKCQGLESRIRHFVSRGRIEHPHLLHKEEIKARRDCADKEENTVVENPSKRVCIIDAEINSIA')]))]),
               OrderedDict([('protein',
                             OrderedDict([('id', 'B'),
                                          ('sequence',
                                       

In [9]:
# Use colab_data_pipeline() as previously to perform the MSA search
mouse_yeast_data = colab_data_pipeline(mouse_yeast)
print_input_json(mouse_yeast_data)

MPLBACKEND=AGG; source /colabfold_venv/bin/activate; colabfold_batch --msa-only --af3-json /tmp/_get_colabfold_msa/input/a98410a49055e01aecca6b9f9deccd9c8a040b79.fasta /tmp/_get_colabfold_msa/output
MPLBACKEND=AGG; source /colabfold_venv/bin/activate; colabfold_batch --msa-only --af3-json /tmp/_get_colabfold_msa/input/87f937a9a60901bd5e6a9eba0437b9b1ff6e3948.fasta /tmp/_get_colabfold_msa/output
{
  "dialect": "alphafold3",
  "version": 2,
  "name": "name",
  "sequences": [
    {
      "protein": {
        "id": "A",
        "sequence": "MAENTDRNQIEKLLNRVKELEQEVERLKKKKEQANNIKDSSIRENSLGSGKAKRAFDFSAHGRRHVALKIAYLGWGYQGFASQENTSNTIEEKLFEALTKTRLVESRQTSNYHRCGRTDKGVSAFGQVISLDLRSQFPTSRDSEDSNLKHEADDLAKEIRYTHILNRVLPADIRVLAWAPVEPSFSARFSCLERTYRYFFPRADLDIATMNYAAQKYVGTHDFRNLCKMDVANGVINFQRTILCAQVQLVAQSPGEERRQEPFQLCQFEVIGQAFLYHQVRCMMAILFLIGQGMEKPEIIDELLNIQKNPQKPQYSMAVEFPLVLYDCKFENTKWIYDHEVQEFNVTHLQQLWANHAVKTHMLYSMLQGLDSVMVTCAAGTKMDEATEWRNIQPPVIKHTSAFVEGVKMRTYKPLMDRPKCQGLESRIRHFVSRGRIEHPHLLHKEEIKARRDCADK

In [10]:
# Set the name attribute & write the mouse-human pair to pairs_input/:
mouse_yeast_data['name'] = 'mouse_yeast'
write_input_json(mouse_yeast_data, 'pairs_input/{}.json')
!ls -l pairs_input/

total 4
-rw-r--r-- 1 root root 1439 Oct  1 12:15 mouse_yeast.json


In [12]:
# Run AlphaFold 3 on pairs_input/ as above
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/pairs_input/ \
  --output_dir=/content/pairs_output/ \
  --norun_data_pipeline

Updated 1 path from the index
Using compilation time workaround with XLA flags
Using low GPU memory setup (40 GB)
Adjusting pair_transition_shard_spec in model_config.py
Enabling unified memory
Executing run_alphafold.py
I1001 12:16:37.637499 139312107823744 xla_bridge.py:895] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
I1001 12:16:37.638558 139312107823744 xla_bridge.py:895] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory

Running AlphaFold 3. Please note that standard AlphaFold 3 model parameters are
only available under terms of use provided at
https://github.com/google-deepmind/alphafold3/blob/main/WEIGHTS_TERMS_OF_USE.md.
If you do not agree to these terms and are using AlphaFold 3 derived model
parameters, cancel execution of AlphaFold 3 inference with CTRL-C, and do not
use the model parameters.

Found local devices: [CudaDevic

In [13]:
# Show output with the confidence metrics
!cat pairs_output/mouse_yeast/mouse_yeast_summary_confidences.json

{
 "chain_iptm": [
  0.15,
  0.15
 ],
 "chain_pair_iptm": [
  [
   0.19,
   0.15
  ],
  [
   0.15,
   0.19
  ]
 ],
 "chain_pair_pae_min": [
  [
   0.76,
   7.68
  ],
  [
   7.34,
   0.76
  ]
 ],
 "chain_ptm": [
  0.19,
  0.19
 ],
 "fraction_disordered": 0.4,
 "has_clash": 0.0,
 "iptm": 0.15,
 "ptm": 0.18,
 "ranking_score": 0.36
}

In [14]:
# Show matrix of interaction-level ipTM confidence metrics:
with open('pairs_output/mouse_yeast/mouse_yeast_summary_confidences.json') as fh:
  summary_confidences = json.load(fh)
print(np.matrix(summary_confidences['chain_pair_iptm']))

[[0.19 0.15]
 [0.15 0.19]]


# 1x3 pool

In [15]:
# Due to time constraints, we'll run one "1x3" pool, e.g. pyrfu-pyrfu-ecoli-yeast:
!cat fasta/pyrfu.fasta
!cat fasta/pyrfu.fasta
!cat fasta/yeast.fasta
!cat fasta/xentr.fasta

>pyrfu
MRVALKIAYDGTKFHGFQRQPNVRTIEGEILRALKDAGIEFENFKSASRTDRGVSARGNV
IALSTEDDRIKNPMVLNSRMSDVWIWGIAEVPEDFHPRFWANTKVYRYYLPSLGMNIKKM
KECSLLFLGTHDFSAFSRVDGRDTIRSIDRIEIWEKCNVVVVEIEGKSFLWEMVRRIVSA
LVLCSQGVLAEERIVEMLNGKFEKSRKVPPAPPEGLLLWDIKYENVEFQIDNASLKKFQR
EIVERFKLHASLSALYEDLILNEQKI
>pyrfu
MRVALKIAYDGTKFHGFQRQPNVRTIEGEILRALKDAGIEFENFKSASRTDRGVSARGNV
IALSTEDDRIKNPMVLNSRMSDVWIWGIAEVPEDFHPRFWANTKVYRYYLPSLGMNIKKM
KECSLLFLGTHDFSAFSRVDGRDTIRSIDRIEIWEKCNVVVVEIEGKSFLWEMVRRIVSA
LVLCSQGVLAEERIVEMLNGKFEKSRKVPPAPPEGLLLWDIKYENVEFQIDNASLKKFQR
EIVERFKLHASLSALYEDLILNEQKI
>yeast
MSNFIRRLVGKMKAISTGTNAIVSKKDSIYANWSKEQLIRRITELENANKPHSEKFQHIE
DNKKRKISQEEVTRSKAKKAPKKFDFSKHNTRFIALRFAYLGWNYNGLAVQKEYTPLPTV
EGTILEAMNKCKLVPSMVLQDYKFSRCGRTDKGVSAMNQVISLEVRSNLTDEEQRDPTND
SREIPYVHVLNQLLPDDIRISAVCLRPPPNFDARFSCVHRHYKYIFNGKNLNIEKMSKAA
SYFVGERDFRNFCKLDGSKQITNFKRTIISSKILPLSETFYCFDLVGSAFLWHQVRCMMA
ILFLVGQSLEVPEIVLRLTDIEKTPQRPVYEMANDIPLLLYDCKFPEMDWQEPTVDDYKA
IKFTTATEALTLHYELKAAVCNIFKDVLPTANTNNFSKTIINLGDGRGKVVGTYVKLEDR
SVMEPVEVVN

In [16]:
# init_input_json() can handle multiple sequences as input
pyrfu_pyrfu_yeast_xentr = init_input_json(
"""
MRVALKIAYDGTKFHGFQRQPNVRTIEGEILRALKDAGIEFENFKSASRTDRGVSARGNV
IALSTEDDRIKNPMVLNSRMSDVWIWGIAEVPEDFHPRFWANTKVYRYYLPSLGMNIKKM
KECSLLFLGTHDFSAFSRVDGRDTIRSIDRIEIWEKCNVVVVEIEGKSFLWEMVRRIVSA
LVLCSQGVLAEERIVEMLNGKFEKSRKVPPAPPEGLLLWDIKYENVEFQIDNASLKKFQR
EIVERFKLHASLSALYEDLILNEQKI""".replace('\n', ''),
"""
MRVALKIAYDGTKFHGFQRQPNVRTIEGEILRALKDAGIEFENFKSASRTDRGVSARGNV
IALSTEDDRIKNPMVLNSRMSDVWIWGIAEVPEDFHPRFWANTKVYRYYLPSLGMNIKKM
KECSLLFLGTHDFSAFSRVDGRDTIRSIDRIEIWEKCNVVVVEIEGKSFLWEMVRRIVSA
LVLCSQGVLAEERIVEMLNGKFEKSRKVPPAPPEGLLLWDIKYENVEFQIDNASLKKFQR
EIVERFKLHASLSALYEDLILNEQKI""".replace('\n', ''),
"""
MSNFIRRLVGKMKAISTGTNAIVSKKDSIYANWSKEQLIRRITELENANKPHSEKFQHIE
DNKKRKISQEEVTRSKAKKAPKKFDFSKHNTRFIALRFAYLGWNYNGLAVQKEYTPLPTV
EGTILEAMNKCKLVPSMVLQDYKFSRCGRTDKGVSAMNQVISLEVRSNLTDEEQRDPTND
SREIPYVHVLNQLLPDDIRISAVCLRPPPNFDARFSCVHRHYKYIFNGKNLNIEKMSKAA
SYFVGERDFRNFCKLDGSKQITNFKRTIISSKILPLSETFYCFDLVGSAFLWHQVRCMMA
ILFLVGQSLEVPEIVLRLTDIEKTPQRPVYEMANDIPLLLYDCKFPEMDWQEPTVDDYKA
IKFTTATEALTLHYELKAAVCNIFKDVLPTANTNNFSKTIINLGDGRGKVVGTYVKLEDR
SVMEPVEVVNAKYSKKKNNKNK""".replace('\n', ''),
"""
MSQEDEVTQLRQRVRELEDKVLRLEEQCRGEAPPQGVVQGPSHTKQKQRLQRAFDFSAHP
KQHVALRLAYLGWGYQGFASQENTKNTVEETVFNALIKTRLVESRQTSNYHRCGRTDRGV
SALGQVISLDLRCSGEGKPEIRYVHILNRVLPPDIRVLSWAAVPPSFSARFSCRSRTYRY
LFPCAQLDVEAMDKAARLLEGTHDFRNLCKMDVANGVMNFIRTVLSARVEPVPEIPGDMG
PFQLYRLQITGLAFLYHQVRCIMGVLFLVGQGKEKPEVITELLDVAKNPRKPQYNMAVEF
PLVLYNCHFDDVQWVSEKEAQSLTVSHLQRLWTQEAVKTQILFMAIGGLDYIPISLETGQ
DLVPWNLCKPTVCDLSGGLVEGVSARSYTPLMKRHLCQGLEERVQHYVKRGRITLPSDRD
RLEDKTPTEKGEKEVADNQVQNNGQLPHSNTQSKGGVKLDQSRDYNKEGAFSQTLERGET
VSESENCDPTTQEKPAKRQCLGTGN""".replace('\n', ''),
)
pyrfu_pyrfu_yeast_xentr

OrderedDict([('dialect', 'alphafold3'),
             ('version', 2),
             ('name', 'name'),
             ('sequences',
              [OrderedDict([('protein',
                             OrderedDict([('id', 'A'),
                                          ('sequence',
                                           'MRVALKIAYDGTKFHGFQRQPNVRTIEGEILRALKDAGIEFENFKSASRTDRGVSARGNVIALSTEDDRIKNPMVLNSRMSDVWIWGIAEVPEDFHPRFWANTKVYRYYLPSLGMNIKKMKECSLLFLGTHDFSAFSRVDGRDTIRSIDRIEIWEKCNVVVVEIEGKSFLWEMVRRIVSALVLCSQGVLAEERIVEMLNGKFEKSRKVPPAPPEGLLLWDIKYENVEFQIDNASLKKFQREIVERFKLHASLSALYEDLILNEQKI')]))]),
               OrderedDict([('protein',
                             OrderedDict([('id', 'B'),
                                          ('sequence',
                                           'MRVALKIAYDGTKFHGFQRQPNVRTIEGEILRALKDAGIEFENFKSASRTDRGVSARGNVIALSTEDDRIKNPMVLNSRMSDVWIWGIAEVPEDFHPRFWANTKVYRYYLPSLGMNIKKMKECSLLFLGTHDFSAFSRVDGRDTIRSIDRIEIWEKCNVVVVEIEGKSFLWEMVRRIVSALVLCSQGVLAEERIVEMLNGKFEKSRKVPP

In [17]:
# Use colab_data_pipeline() as previously to attach the MSAs
pyrfu_pyrfu_yeast_xentr_data = colab_data_pipeline(pyrfu_pyrfu_yeast_xentr)

MPLBACKEND=AGG; source /colabfold_venv/bin/activate; colabfold_batch --msa-only --af3-json /tmp/_get_colabfold_msa/input/2ac1642aefd62bfd7e970102a5cb8a3dac21148d.fasta /tmp/_get_colabfold_msa/output
MPLBACKEND=AGG; source /colabfold_venv/bin/activate; colabfold_batch --msa-only --af3-json /tmp/_get_colabfold_msa/input/9ff98a507ef190ecaccd07e7f543568bc0182e79.fasta /tmp/_get_colabfold_msa/output


In [18]:
# Set the name attribute & write the pool to pools_input/:
pyrfu_pyrfu_yeast_xentr_data['name'] = 'pyrfu_pyrfu_yeast_xentr'
write_input_json(pyrfu_pyrfu_yeast_xentr_data, 'pools_input/{}.json')
!ls -l pools_input/

total 18216
-rw-r--r-- 1 root root 18653178 Oct  1 12:22 pyrfu_pyrfu_yeast_xentr.json


In [19]:
# Run AlphaFold 3 on pools_input/
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/pools_input/ \
  --output_dir=/content/pools_output/ \
  --norun_data_pipeline

Updated 1 path from the index
Using compilation time workaround with XLA flags
Using low GPU memory setup (40 GB)
Adjusting pair_transition_shard_spec in model_config.py
Enabling unified memory
Executing run_alphafold.py
I1001 12:22:49.476992 138398294884992 xla_bridge.py:895] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
I1001 12:22:49.478073 138398294884992 xla_bridge.py:895] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory

Running AlphaFold 3. Please note that standard AlphaFold 3 model parameters are
only available under terms of use provided at
https://github.com/google-deepmind/alphafold3/blob/main/WEIGHTS_TERMS_OF_USE.md.
If you do not agree to these terms and are using AlphaFold 3 derived model
parameters, cancel execution of AlphaFold 3 inference with CTRL-C, and do not
use the model parameters.

Found local devices: [CudaDevic

In [20]:
# Show matrix of interaction-level ipTM confidence metrics
with open('pools_output/pyrfu_pyrfu_yeast_xentr/pyrfu_pyrfu_yeast_xentr_summary_confidences.json') as fh:
  summary_confidences = json.load(fh)
print(np.matrix(summary_confidences['chain_pair_iptm']))

[[0.88 0.71 0.09 0.1 ]
 [0.71 0.88 0.09 0.1 ]
 [0.09 0.09 0.65 0.26]
 [0.1  0.1  0.26 0.61]]
